#  DataBase contruction

In [3]:
import pandas as pd

In [9]:
# reading all the data files
data_clinical_patient = pd.read_csv('pan_origimed_2020/data_clinical_patient.txt', sep="\t")
data_clinical_sample = pd.read_csv('pan_origimed_2020/data_clinical_sample.txt', sep="\t")
data_cna_log2 = pd.read_csv('pan_origimed_2020/data_cna_log2.txt', sep="\t")
data_cna = pd.read_csv('pan_origimed_2020/data_cna.txt', sep="\t")
data_mutations = pd.read_csv('pan_origimed_2020/data_mutations.txt', sep="\t", header=2, dtype={"Exon_Number": "string"})
data_sv = pd.read_csv('pan_origimed_2020/data_sv.txt', sep="\t")

In [10]:
# removing bad rows
data_clinical_sample = data_clinical_sample[4:]
data_clinical_patient = data_clinical_patient[4:]

In [15]:
data_clinical_patient = data_clinical_patient.rename(columns={'#Patient Identifier': 'PATIENT_ID'})

In [18]:
data_clinical_patient.head()

,PATIENT_ID,Sex,Diagnosis Age,Smoke Status,Treatment,SAMPLE_ID
4,Patient0001,Female,67,Unknown,Other_Treatments,P-0001
5,Patient0002,Male,75,Unknown,Treatment-naive,P-0002
6,Patient0003,Female,45,Unknown,Treatment-naive,P-0003
7,Patient0004,Male,70,Unknown,Treatment-naive,P-0004
8,Patient0005,Male,53,Unknown,Treatment-naive,P-0005


In [17]:
# matching the sample id to match other tables
data_clinical_patient["SAMPLE_ID"] = data_clinical_patient["PATIENT_ID"].apply(lambda x: "P-" + x[7:])

In [19]:
# make all sample id header name the same - "SAMPLE_ID"
data_clinical_sample.rename(columns={"Sample Identifier": 'SAMPLE_ID'}, inplace=True)
data_mutations.rename(columns={"Tumor_Sample_Barcode": 'SAMPLE_ID'}, inplace=True)
data_sv.rename(columns={"Sample_Id": 'SAMPLE_ID'}, inplace=True)

In [20]:
# merge everything
merged_clinical_data = data_clinical_patient.merge(data_clinical_sample, on="SAMPLE_ID", how='outer')
merged_mutations_data = merged_clinical_data.merge(data_mutations, on="SAMPLE_ID", how='outer')
merged_all_data = merged_mutations_data.merge(data_sv, on="SAMPLE_ID", how='outer')

In [21]:
merged_all_data.to_csv("pan_cancer_db_merged.csv")

In [22]:
merged_all_data["SNP_event"] = merged_all_data["Reference_Allele"].fillna("").astype(str) + ">" + merged_all_data["Tumor_Seq_Allele2"].fillna("").astype(str)


In [24]:
merged_all_data.head()

,PATIENT_ID,Sex,Diagnosis Age,Smoke Status,Treatment,SAMPLE_ID,#Patient Identifier,Cancer Type,Cancer Type Detailed,Tumor Stage,...,Site2_Hugo_Symbol,Center_y,Event_Info,DNA_support,RNA_support,Method,Connection_Type,SV_Status,Group,SNP_event
0,Patient0001,Female,67,Unknown,Other_Treatments,P-0001,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C>T
1,Patient0001,Female,67,Unknown,Other_Treatments,P-0001,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C>A
2,Patient0001,Female,67,Unknown,Other_Treatments,P-0001,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C>-
3,Patient0001,Female,67,Unknown,Other_Treatments,P-0001,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C>T
4,Patient0001,Female,67,Unknown,Other_Treatments,P-0001,Patient0001,Colorectal Carcinoma,Colorectal Adenocarcinoma,IV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G>T


In [26]:
data_for_model = merged_all_data[["PATIENT_ID", "Sex", "Diagnosis Age", "Smoke Status", "TMB (nonsynonymous)",
                                "Hugo_Symbol", "Chromosome", "Start_Position", "End_Position",
                                "Consequence", "Variant_Type", "SNP_event", "Protein_position", "Codons",
                                "Exon_Number","VAR_TYPE_SX", "Site1_Hugo_Symbol", "Site2_Hugo_Symbol","Event_Info"]]


In [266]:
data_for_model.to_csv("pan_cancer_db_for_model.csv")

In [27]:
data_for_model.shape

(105906, 19)

In [28]:
data_for_model.head(20)

,PATIENT_ID,Sex,Diagnosis Age,Smoke Status,TMB (nonsynonymous),Hugo_Symbol,Chromosome,Start_Position,End_Position,Consequence,Variant_Type,SNP_event,Protein_position,Codons,Exon_Number,VAR_TYPE_SX,Site1_Hugo_Symbol,Site2_Hugo_Symbol,Event_Info
0,Patient0001,Female,67,Unknown,0.333333333,KMT2C,7,151836340.0,151836340.0,missense_variant,SNP,C>T,4822.0,cGt/cAt,57/59,Substitution/Indel,NaN,NaN,NaN
1,Patient0001,Female,67,Unknown,0.333333333,KRAS,12,25398285.0,25398285.0,missense_variant,SNP,C>A,12.0,Ggt/Tgt,5-Feb,Substitution/Indel,NaN,NaN,NaN
2,Patient0001,Female,67,Unknown,0.333333333,SOX9,17,70119705.0,70119705.0,frameshift_variant,DEL,C>-,236.0,aCc/ac,3-Mar,Truncation,NaN,NaN,NaN
3,Patient0001,Female,67,Unknown,0.333333333,APC,5,112128143.0,112128143.0,"stop_gained,splice_region_variant",SNP,C>T,216.0,Cga/Tga,16-Jul,Truncation,NaN,NaN,NaN
4,Patient0001,Female,67,Unknown,0.333333333,APC,5,112175147.0,112175147.0,stop_gained,SNP,G>T,1286.0,Gaa/Taa,16/16,Truncation,NaN,NaN,NaN
5,Patient0001,Female,67,Unknown,0.333333333,QKI,6,163836364.0,163836364.0,missense_variant,SNP,G>A,47.0,Gaa/Aaa,8-Jan,Substitution/Indel,NaN,NaN,NaN
6,Patient0001,Female,67,Unknown,0.333333333,FBXW7,4,153247289.0,153247289.0,missense_variant,SNP,G>A,505.0,Cgc/Tgc,12-Oct,Substitution/Indel,NaN,NaN,NaN
7,Patient0001,Female,67,Unknown,0.333333333,PTPN11,12,112910733.0,112910736.0,intron_variant,DEL,TTTC>-,NaN,NaN,<NA>,Substitution/Indel,NaN,NaN,NaN
8,Patient0001,Female,67,Unknown,0.333333333,GLI1,12,57863367.0,57863367.0,missense_variant,SNP,G>T,488.0,Gct/Tct,12-Nov,Substitution/Indel,NaN,NaN,NaN
9,Patient0001,Female,67,Unknown,0.333333333,ATM,11,108142138.0,108142138.0,"splice_region_variant,intron_variant",SNP,G>A,1026.0,NaN,<NA>,Substitution/Indel,NaN,NaN,NaN


In [269]:
# checking if the dates in Exon number are dd/mm or mm/dd by comparing the dates and not dates for gene APC
# data_for_model[data_for_model["Hugo_Symbol"].str.contains("APC", na=False)]
# result - the format of the dates are total exons-exon number

In [29]:
# Function to handle the conversion
def convert_exon_number(val):
    try:
        # First, try to convert to 'Month-Year' format (e.g., 'Sep-89' -> '09/89')
        return pd.to_datetime(val, format='%b-%y').strftime('%m/%y')
    except ValueError:
        pass

    try:
        # Then, try to convert to 'DD-Mon' format (e.g., '14-Sep' -> '09/14')
        date_obj = pd.to_datetime(val, format='%d-%b', errors='raise')
        return date_obj.strftime('%m/%d')
    except ValueError:
        # If neither format matches, return the value as is (non-date-like string)
        return val

In [31]:
# Apply the function to the column
data_for_model.loc[:, 'Exon_Number'] = data_for_model['Exon_Number'].apply(convert_exon_number)

In [32]:
data_for_model.head(10)

,PATIENT_ID,Sex,Diagnosis Age,Smoke Status,TMB (nonsynonymous),Hugo_Symbol,Chromosome,Start_Position,End_Position,Consequence,Variant_Type,SNP_event,Protein_position,Codons,Exon_Number,VAR_TYPE_SX,Site1_Hugo_Symbol,Site2_Hugo_Symbol,Event_Info
0,Patient0001,Female,67,Unknown,0.333333333,KMT2C,7,151836340.0,151836340.0,missense_variant,SNP,C>T,4822.0,cGt/cAt,57/59,Substitution/Indel,NaN,NaN,NaN
1,Patient0001,Female,67,Unknown,0.333333333,KRAS,12,25398285.0,25398285.0,missense_variant,SNP,C>A,12.0,Ggt/Tgt,02/05,Substitution/Indel,NaN,NaN,NaN
2,Patient0001,Female,67,Unknown,0.333333333,SOX9,17,70119705.0,70119705.0,frameshift_variant,DEL,C>-,236.0,aCc/ac,03/03,Truncation,NaN,NaN,NaN
3,Patient0001,Female,67,Unknown,0.333333333,APC,5,112128143.0,112128143.0,"stop_gained,splice_region_variant",SNP,C>T,216.0,Cga/Tga,07/16,Truncation,NaN,NaN,NaN
4,Patient0001,Female,67,Unknown,0.333333333,APC,5,112175147.0,112175147.0,stop_gained,SNP,G>T,1286.0,Gaa/Taa,16/16,Truncation,NaN,NaN,NaN
5,Patient0001,Female,67,Unknown,0.333333333,QKI,6,163836364.0,163836364.0,missense_variant,SNP,G>A,47.0,Gaa/Aaa,01/08,Substitution/Indel,NaN,NaN,NaN
6,Patient0001,Female,67,Unknown,0.333333333,FBXW7,4,153247289.0,153247289.0,missense_variant,SNP,G>A,505.0,Cgc/Tgc,10/12,Substitution/Indel,NaN,NaN,NaN
7,Patient0001,Female,67,Unknown,0.333333333,PTPN11,12,112910733.0,112910736.0,intron_variant,DEL,TTTC>-,NaN,NaN,<NA>,Substitution/Indel,NaN,NaN,NaN
8,Patient0001,Female,67,Unknown,0.333333333,GLI1,12,57863367.0,57863367.0,missense_variant,SNP,G>T,488.0,Gct/Tct,11/12,Substitution/Indel,NaN,NaN,NaN
9,Patient0001,Female,67,Unknown,0.333333333,ATM,11,108142138.0,108142138.0,"splice_region_variant,intron_variant",SNP,G>A,1026.0,NaN,<NA>,Substitution/Indel,NaN,NaN,NaN


In [33]:
data_for_model["Hugo_Symbol"].value_counts()

Hugo_Symbol
TP53            6719
LRP1B           2138
APC             1882
KRAS            1781
EGFR            1402
                ... 
STK24-AS1          1
EGFR-AS1           1
DPYD-AS1           1
MIR4466            1
RP11-770J1.3       1
Name: count, Length: 479, dtype: int64

In [274]:
data_for_model["Chromosome"].value_counts()

Chromosome
17    11798
2      8468
1      7888
7      7764
3      7694
12     7669
5      6356
4      5822
8      4713
19     4491
X      4341
11     4282
9      3657
16     3449
6      3194
13     2753
10     2587
20     1711
22     1663
14     1648
15     1583
18     1516
21      446
Name: count, dtype: int64

In [275]:
data_for_model["Consequence"].value_counts()


Consequence
missense_variant                                                                      65649
frameshift_variant                                                                    10367
stop_gained                                                                            8673
intron_variant                                                                         5213
splice_region_variant,intron_variant                                                   5136
missense_variant,splice_region_variant                                                 1916
splice_acceptor_variant                                                                1725
inframe_deletion                                                                       1415
splice_donor_variant                                                                   1354
upstream_gene_variant                                                                  1214
splice_region_variant,synonymous_variant                            

In [34]:
data_for_model['Consequence'].str.split(',')
dummy_vars = data_for_model['Consequence'].str.split(',').explode().str.get_dummies().groupby(level=0).sum()
data_for_model = data_for_model.join(dummy_vars)


In [35]:
data_for_model['Exon_Number'].value_counts()

Exon_Number
02/05    2012
05/11    1835
07/11    1565
08/11    1510
16/16    1468
         ... 
28/51       1
25/45       1
46/87       1
01/47       1
03/58       1
Name: count, Length: 1848, dtype: int64

In [36]:
data_for_model.head(50)

,PATIENT_ID,Sex,Diagnosis Age,Smoke Status,TMB (nonsynonymous),Hugo_Symbol,Chromosome,Start_Position,End_Position,Consequence,...,splice_acceptor_variant,splice_donor_variant,splice_region_variant,start_lost,start_retained_variant,stop_gained,stop_lost,stop_retained_variant,synonymous_variant,upstream_gene_variant
0,Patient0001,Female,67,Unknown,0.333333333,KMT2C,7,151836340.0,151836340.0,missense_variant,...,0,0,0,0,0,0,0,0,0,0
1,Patient0001,Female,67,Unknown,0.333333333,KRAS,12,25398285.0,25398285.0,missense_variant,...,0,0,0,0,0,0,0,0,0,0
2,Patient0001,Female,67,Unknown,0.333333333,SOX9,17,70119705.0,70119705.0,frameshift_variant,...,0,0,0,0,0,0,0,0,0,0
3,Patient0001,Female,67,Unknown,0.333333333,APC,5,112128143.0,112128143.0,"stop_gained,splice_region_variant",...,0,0,1,0,0,1,0,0,0,0
4,Patient0001,Female,67,Unknown,0.333333333,APC,5,112175147.0,112175147.0,stop_gained,...,0,0,0,0,0,1,0,0,0,0
5,Patient0001,Female,67,Unknown,0.333333333,QKI,6,163836364.0,163836364.0,missense_variant,...,0,0,0,0,0,0,0,0,0,0
6,Patient0001,Female,67,Unknown,0.333333333,FBXW7,4,153247289.0,153247289.0,missense_variant,...,0,0,0,0,0,0,0,0,0,0
7,Patient0001,Female,67,Unknown,0.333333333,PTPN11,12,112910733.0,112910736.0,intron_variant,...,0,0,0,0,0,0,0,0,0,0
8,Patient0001,Female,67,Unknown,0.333333333,GLI1,12,57863367.0,57863367.0,missense_variant,...,0,0,0,0,0,0,0,0,0,0
9,Patient0001,Female,67,Unknown,0.333333333,ATM,11,108142138.0,108142138.0,"splice_region_variant,intron_variant",...,0,0,1,0,0,0,0,0,0,0


In [37]:
data_for_model = pd.get_dummies(data_for_model, columns=['Smoke Status'], drop_first=False)
data_for_model.drop('Smoke Status_Unknown', axis=1, inplace=True)

In [46]:
data_for_model.to_csv("pan_cancer_data_for_model.csv", index=False)